In [54]:
# theme = 'plotly_dark'
theme = 'seaborn'
#theme = 'plotly'

In [55]:
import pandas as pd
import json
import torch



import plotly.express as px
# import plotly.graph_objects as go
# import plotly.subplots as sp
import os

In [56]:
PROJECT_DIR = '/home/kuba/Documents/Data/Raw/Listerine/3_final'  
watch_num = '03'
recording = '2023-07-18_07_21_53'

HERTZ = 100
ACTIVITY_NAME_TO_CLASS_INDEX_MAPPING = {
    'water':0,
    'listerine':1,
}

In [57]:
#takes in label a to tensor for ML
def json_to_tensor(labels_x, acc_len_x, acc_x):
    y_new = torch.zeros(acc_len_x)-1

    bouts = []
    for hand in labels_x:
        for action in labels_x[hand]:
            for bout in labels_x[hand][action]:
                y_new[(acc_x.timestamp > bout['start']) & (acc_x.timestamp < bout['end'])] = (ACTIVITY_NAME_TO_CLASS_INDEX_MAPPING[action] * 20 + 15)
    return y_new



In [58]:
def viz_labels(df_acc, y, title):
    y_df = pd.DataFrame(y,columns=['labels'])
    df = pd.concat([df_acc, y_df], axis=1)
    df = df[::5]
    fig = px.line(df, x='timestamp', y=['x','y','z','labels'], title=title)
    fig.update_layout(template=theme)
    fig.show(renderer='browser')

In [59]:
WATCH_DIR = f'{PROJECT_DIR}/{watch_num}'

if recording == '*':
    for recording in sorted(os.listdir(WATCH_DIR)):
        print(recording)
        if recording == '.DS_Store':
            continue
        recording_dir = f'{WATCH_DIR}/{recording}'
        acc = pd.read_csv(f'{recording_dir}/acceleration.csv',skiprows=1)
        with open(f'{recording_dir}/labels.json','r') as f:
            labels = json.load(f)  
        acc_len = len(acc)
        acc['timestamp'] = ((acc['timestamp'] - acc['timestamp'].iloc[0]) * 1e-9)
        y = json_to_tensor(labels, acc_len, acc)
        viz_labels(acc, y, recording_dir)
else:
    recording_dir = f'{WATCH_DIR}/{recording}'
    acc = pd.read_csv(f'{recording_dir}/acceleration.csv',skiprows=1)
    with open(f'{recording_dir}/labels.json','r') as f:
        labels = json.load(f)  
    acc_len = len(acc)
    acc['timestamp'] = ((acc['timestamp'] - acc['timestamp'].iloc[0]) * 1e-9)
    y = json_to_tensor(labels, acc_len, acc)
    viz_labels(acc, y, recording_dir)





In [60]:
def viz_labels_and_predictions(df_acc, y, model, window_size, stride, device, title):
    # Prepare the data for visualization
    y_df = pd.DataFrame(y, columns=['labels'])
    df = pd.concat([df_acc, y_df], axis=1)
    
    # Get predictions for each window
    predictions = []
    timestamps = []
    
    # Create windows of data
    for i in range(0, len(df) - window_size + 1, stride):
        window = df.iloc[i:i + window_size]
        
        # Prepare data for model
        X_acc = torch.tensor(window[['x', 'y', 'z']].values, dtype=torch.float32)
        X_acc = X_acc.unsqueeze(0)  # Add batch dimension
        X_acc = X_acc.to(device)
        
        # Get prediction
        with torch.no_grad():
            output = model(X_acc)
            pred = torch.sigmoid(output) > 0.5
            predictions.append(pred.item())
            # Use the middle timestamp of the window
            timestamps.append(window['timestamp'].iloc[window_size//2])
    
    # Create prediction DataFrame
    pred_df = pd.DataFrame({
        'timestamp': timestamps,
        'predictions': predictions
    })
    
    # Create the visualization
    fig = go.Figure()
    
    # Plot accelerometer data
    for col in ['x', 'y', 'z']:
        fig.add_trace(go.Scatter(
            x=df['timestamp'],
            y=df[col],
            name=col,
            mode='lines',
            opacity=0.7
        ))
    
    # Plot true labels
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['labels'],
        name='True Labels',
        mode='lines',
        line=dict(color='black', width=2)
    ))
    
    # Plot predictions
    fig.add_trace(go.Scatter(
        x=pred_df['timestamp'],
        y=pred_df['predictions'],
        name='Predictions',
        mode='lines',
        line=dict(color='red', width=2, dash='dash')
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Time (s)',
        yaxis_title='Value',
        template=theme,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    fig.show(renderer='browser')

In [61]:
def read_txt(dir):
    txt_path = os.path.join(dir, f'desc.txt')
    with open(txt_path, 'r') as f:
        content = f.read()
    dic = eval(content)
    return dic['window_size'], dic['stride'] 

In [64]:
import sys
sys.path.append('/home/kuba/Projects/MedicationTakingData/resmodel')


mode_dir = '/home/kuba/Projects/MedicationTakingData/resmodel/res_search_00/res_search_00_7'
head_tail = os.path.split(mode_dir)
# model_path = os.path.join(mode_dir, f'{head_tail[1]}_bestLoss.pth')
model_path = os.path.join(mode_dir, f'{head_tail[1]}_bestF1.pth')

WATCH_DIR = '/home/kuba/Documents/Data/Raw/Listerine/3_final/03'
recording = '2023-07-18_07_21_53'

window_size, stride = read_txt(mode_dir)

print(model_path)
model = torch.load(model_path)
model.eval()
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)


if recording == '*':
    recordings = sorted(os.listdir(WATCH_DIR))
else:
    recordings = [recording]
    
for rec in recordings:
    if rec == '.DS_Store':
        continue
        
    print(f"Processing recording: {rec}")
    recording_dir = f'{WATCH_DIR}/{rec}'
    
    # Load acceleration data
    acc = pd.read_csv(f'{recording_dir}/acceleration.csv', skiprows=1)
    acc['timestamp'] = (acc['timestamp'] - acc['timestamp'].iloc[0]) * 1e-9
    
    # Load labels
    with open(f'{recording_dir}/labels.json', 'r') as f:
        labels = json.load(f)
        
    # Convert labels to tensor
    acc_len = len(acc)
    y = json_to_tensor(labels, acc_len, acc)
    
    # Visualize with predictions
    viz_labels_and_predictions(
        acc, 
        y, 
        model,
        window_size=window_size,  # adjust as needed
        stride=stride,        # adjust as needed
        device=device,    # make sure device is defined
        title=recording_dir
    )


/home/kuba/Projects/MedicationTakingData/resmodel/res_search_00/res_search_00_7/res_search_00_7_bestF1.pth
Using device: cuda:1
Processing recording: 2023-07-18_07_21_53


RuntimeError: Given groups=1, weight of size [12, 6, 1], expected input[1, 305, 3] to have 6 channels, but got 305 channels instead